# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# pip install gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,10.40,37,20,3.44,GL,1612521375
1,1,meulaboh,4.1363,96.1285,83.79,70,72,4.34,ID,1612521375
2,2,thompson,55.7435,-97.8558,-18.40,76,40,6.91,CA,1612521375
3,3,bandarbeyla,9.4942,50.8122,77.27,60,12,12.57,SO,1612521375
4,4,constitucion,-35.3333,-72.4167,58.42,87,94,4.18,CL,1612521375
...,...,...,...,...,...,...,...,...,...,...
554,554,kieta,-6.2167,155.6333,81.01,79,95,6.40,PG,1612521432
555,555,grand gaube,-20.0064,57.6608,87.01,62,0,8.01,MU,1612521432
556,556,cedar city,37.6775,-113.0619,21.20,68,1,2.33,US,1612521348
557,557,misratah,32.3754,15.0925,70.30,35,24,10.20,LY,1612521433


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_data[["Lat", "lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_city_df = weather_data.loc[(weather_data["Max Temp"] <=80) & (weather_data["Max Temp"] >=70)].dropna()
narrow_city_df                               

,City ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,bandarbeyla,9.4942,50.8122,77.27,60,12,12.57,SO,1612521375
5,5,hobart,-42.8794,147.3294,71.01,88,75,10.36,AU,1612521375
8,8,cape town,-33.9258,18.4232,78.01,57,20,13.80,ZA,1612521375
9,9,namibe,-15.1961,12.1522,79.81,54,15,13.13,AO,1612521376
12,12,rikitea,-23.1203,-134.9692,77.23,75,57,14.14,PF,1612521376
...,...,...,...,...,...,...,...,...,...,...
542,542,saquarema,-22.9200,-42.5103,73.40,100,90,4.61,BR,1612521431
546,546,aden,12.7794,45.0367,76.35,66,15,14.34,YE,1612521344
551,551,bonfim,3.0833,-59.9500,75.09,79,100,9.60,BR,1612521432
557,557,misratah,32.3754,15.0925,70.30,35,24,10.20,LY,1612521433


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "lng", "Max Temp"]]

hotel_df["Hotel Name"] = ""

hotel_df


,City,Country,Lat,lng,Max Temp,Hotel Name
3,bandarbeyla,SO,9.4942,50.8122,77.27,
5,hobart,AU,-42.8794,147.3294,71.01,
8,cape town,ZA,-33.9258,18.4232,78.01,
9,namibe,AO,-15.1961,12.1522,79.81,
12,rikitea,PF,-23.1203,-134.9692,77.23,
...,...,...,...,...,...,...
542,saquarema,BR,-22.9200,-42.5103,73.40,
546,aden,YE,12.7794,45.0367,76.35,
551,bonfim,BR,3.0833,-59.9500,75.09,
557,misratah,LY,32.3754,15.0925,70.30,


In [7]:
endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [12]:

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from hotel_df
    lat = row["Lat"]
    lng = row["lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Getting the Results for Index {index}: '{city_name}'.")
    response = requests.get(endpoint_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Getting the Results for Index 3: 'bandarbeyla'.
Closest hotel in bandarbeyla is JABIR HOTEL.
------------
Getting the Results for Index 5: 'hobart'.
Closest hotel in hobart is Hotel Grand Chancellor Hobart.
------------
Getting the Results for Index 8: 'cape town'.
Closest hotel in cape town is Atlanticview Cape Town Boutique Hotel.
------------
Getting the Results for Index 9: 'namibe'.
Closest hotel in namibe is iu Hotel Namibe.
------------
Getting the Results for Index 12: 'rikitea'.
Closest hotel in rikitea is Pension Bianca & Benoit.
------------
Getting the Results for Index 14: 'dingle'.
Missing field/result... skipping.
------------
Getting the Results for Index 17: 'bubaque'.



KeyboardInterrupt



In [10]:
hotel_df

,City,Country,Lat,lng,Max Temp,Hotel Name
3,bandarbeyla,SO,9.4942,50.8122,77.27,JABIR HOTEL
5,hobart,AU,-42.8794,147.3294,71.01,Hotel Grand Chancellor Hobart
8,cape town,ZA,-33.9258,18.4232,78.01,Atlanticview Cape Town Boutique Hotel
9,namibe,AO,-15.1961,12.1522,79.81,iu Hotel Namibe
12,rikitea,PF,-23.1203,-134.9692,77.23,Pension Bianca & Benoit
...,...,...,...,...,...,...
542,saquarema,BR,-22.9200,-42.5103,73.40,Hotel Maasai
546,aden,YE,12.7794,45.0367,76.35,Sheraton Gold Mohur Hotel
551,bonfim,BR,3.0833,-59.9500,75.09,
557,misratah,LY,32.3754,15.0925,70.30,نزل طريق المطار - مصراتة


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))